In [ ]:
import pandas as pd
import numpy as np
from sklearn.impute import KNNImputer
from sklearn.preprocessing import MinMaxScaler

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
features_and_target_with_nans = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/예비실험 4/예비실험4 - Extract & Imputation/features_target_combined_with_nans.csv')

In [ ]:
features_and_target_with_nans = features_and_target_with_nans.drop(columns = ['Unnamed: 0'])

In [ ]:
data = features_and_target_with_nans[['SubjectID', 'Gender', 'Race', 'family_ALS_hist', 'if_use_Riluzole', 'treatment_group', 'onset_site']]

In [ ]:
data1 = data.copy()

In [ ]:
mm = MinMaxScaler()

def find_category_mappings(data, variable):
    return {k: i for i, k in enumerate(data[variable].dropna().unique(), 0)}

def integer_encode(df , variable, ordinal_mapping):
    data[variable] = data[variable].map(ordinal_mapping)

mappin = dict()
def imputation(data1 , cols):
    data = data1.copy()
    #Encoding dict &amp; Removing nan    
    #mappin = dict()
    for variable in cols:
        mappings = find_category_mappings(data, variable)
        mappin[variable] = mappings

    #Apply mapping
    for variable in cols:
        integer_encode(data, variable, mappin[variable])  

    #Minmaxscaler and KNN imputation 
    sca = mm.fit_transform(data)
    knn_imputer = KNNImputer()
    knn = knn_imputer.fit_transform(sca)
    data.iloc[:,:] = mm.inverse_transform(knn)
    for i in data.columns : 
        data[i] = round(data[i]).astype('int')

    #Inverse transform
    for i in cols:
        inv_map = {v: k for k, v in mappin[i].items()}
        data[i] = data[i].map(inv_map)
    return data


#Imputation
data_imp = imputation(data,['Gender', 'Race', 'family_ALS_hist', 'if_use_Riluzole', 'treatment_group', 'onset_site'])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [ ]:
data_imp

,SubjectID,Gender,Race,family_ALS_hist,if_use_Riluzole,treatment_group,onset_site
0,348,0,0,0.0,0.0,0.0,0.0
1,533,0,0,0.0,0.0,0.0,1.0
2,649,0,0,0.0,0.0,1.0,2.0
3,1064,0,0,0.0,0.0,0.0,1.0
4,1234,1,0,0.0,0.0,1.0,2.0
...,...,...,...,...,...,...,...
3091,997136,1,0,0.0,0.0,1.0,2.0
3092,998047,0,0,0.0,0.0,1.0,2.0
3093,998773,1,0,0.0,1.0,1.0,2.0
3094,998908,1,0,0.0,1.0,1.0,2.0


In [ ]:
Gend = {0:"F", 1:"M"}
race = {0:"White", 1:"Black", 2:"Hispanic", 3:"Unknown", 4:"Other",5:"Asian"}
hist = {0:"Yes", 1:"No"}
rilz = {0:"Yes", 1:"No"}
onsetsite = {0:"Other", 1:"Bulbar", 2:"Limb", 3:"Limb and Bulbar"}
treatmentgroup = {0:"Placebo", 1: "Active"}

In [ ]:
data1 = data_imp.copy()

In [ ]:
data1 = data1.replace({'Gender':Gend})
data1 = data1.replace({'Race':race})
data1 = data1.replace({'family_ALS_hist':hist})
data1 = data1.replace({'onset_site':onsetsite})
data1 = data1.replace({'treatment_group':treatmentgroup})
data1 = data1.replace({'if_use_Riluzole':rilz})
data1

,SubjectID,Gender,Race,family_ALS_hist,if_use_Riluzole,treatment_group,onset_site
0,348,F,White,Yes,Yes,Placebo,Other
1,533,F,White,Yes,Yes,Placebo,Bulbar
2,649,F,White,Yes,Yes,Active,Limb
3,1064,F,White,Yes,Yes,Placebo,Bulbar
4,1234,M,White,Yes,Yes,Active,Limb
...,...,...,...,...,...,...,...
3091,997136,M,White,Yes,Yes,Active,Limb
3092,998047,F,White,Yes,Yes,Active,Limb
3093,998773,M,White,Yes,No,Active,Limb
3094,998908,M,White,Yes,No,Active,Limb


In [ ]:
data1.to_csv('/content/drive/MyDrive/Colab Notebooks/예비실험 4/예비실험4 - Extract & Imputation/categorical features without nans.csv')

In [ ]:
num_features2 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/예비실험 4/예비실험4 - Extract & Imputation/numeric_imputed_features_combined.csv')

In [ ]:
num_features2.drop(columns = ['Unnamed: 0'], inplace=True)

In [ ]:
cat_features = data1.copy()

In [ ]:
all_feature_without_nans = pd.concat([data_imp, num_features2], axis=1)

In [ ]:
no_id_only_features = all_feature_without_nans.drop(columns = ['SubjectID'])
target = features_and_target_with_nans[['ALSFRS_slope']]

In [ ]:
feature_and_target_for_MI = pd.concat([no_id_only_features, target], axis=1)

In [ ]:
X

,Gender,Race,family_ALS_hist,if_use_Riluzole,treatment_group,onset_site,Age,diag_delta,onset_delta,alsfrs_total_slope,mean_alsfrs_total,min_alsfrs_total,max_alsfrs_total,mean_alsfrs_mouth,max_alsfrs_mouth,min_alsfrs_mouth,mean_alsfrs_hands,max_alsfrs_hands,min_alsfrs_hands,mean_alsfrs_leg,max_alsfrs_leg,min_alsfrs_leg,mean_alsfrs_trunk,max_alsfrs_trunk,min_alsfrs_trunk,mean_alsfrs_respiratory,max_alsfrs_respiratory,min_alsfrs_respiratory,alsfrs_mouth_slope,alsfrs_hands_slope,alsfrs_leg_slope,alsfrs_trunk_slope,alsfrs_respiratory_slope,fvc_slope,fvc_mean,fvc_min,fvc_max,svc_slope,svc_mean,svc_min,...,resp_rate_slope,temp_slope,mean_weight,max_weight,min_weight,slope_weight,mean_BMI,mean_ALC,max_ALC,min_ALC,mean_AMC,max_AMC,min_AMC,mean_ANC,max_ANC,min_ANC,mean_CK,max_CK,min_CK,mean_Creatinine,max_Creatinine,min_Creatinine,mean_Total_Cholesterol,max_Total_Cholesterol,min_Total_Cholesterol,mean_Triglycerides,max_Triglycerides,min_Triglycerides,mean_Uric_Acid,max_Uric_Acid,min_Uric_Acid,ALC_slope,AMC_slope,ANC_slope,CK_slope,Creaetinine_slope,Total_Cholesterol_slope,Triglycerides_slope,Uric_Acid_slope,ALSFRS_slope
0,0,0,0.0,0.0,0.0,0.0,52.0,-302.000000,-501.000000,-0.009689,30.250000,30.0,31.0,12.000000,12.0,12.0,5.500000,6.0,5.0,3.000000,3.0,3.0,5.750000,7.0,5.0,4.00,4.0,4.0,0.000000,0.016393,0.000000,-0.032787,0.000000,0.284522,86.424228,96.323526,80.35385,-0.018609,81.666667,81.000000,...,-0.065574,0.000000,120.033333,122.5,118.8,0.060656,0.004471,1.781389,1.893045,1.565840,0.385000,0.480000,0.280000,5.210000,6.160000,4.350000,199.750000,261.0,125.0,53.040000,53.040,53.04,4.971600,5.249600,4.861700,1.366100,1.63710,1.15160,295.913000,344.984000,267.660000,-0.003171,0.000889,-0.009667,1.200000,0.000000,-0.004310,0.005394,0.396533,-0.341760
1,0,0,0.0,0.0,0.0,1.0,65.0,-44.000000,-1023.000000,0.002114,28.666667,27.0,30.0,4.333333,5.0,3.0,8.000000,8.0,8.0,6.000000,8.0,4.0,7.333333,8.0,7.0,3.00,3.0,3.0,-0.021978,0.000000,0.021978,-0.010989,0.000000,-0.583333,73.500000,70.000000,77.00000,0.224940,59.909899,68.753669,...,0.000000,-0.003297,51.000000,52.0,50.5,-0.005495,0.003045,1.304147,1.470141,1.245963,0.457718,0.511753,0.425764,5.484587,4.991396,4.994487,149.000000,180.0,118.0,79.560000,79.560,79.56,6.018979,6.449237,5.846188,1.086069,1.04675,1.27705,318.701206,326.818620,310.598938,0.008373,0.004311,0.040468,0.681319,0.000000,0.002109,0.017275,-0.838279,-0.965608
2,0,0,0.0,0.0,1.0,2.0,48.0,-305.367039,-395.010096,-0.000537,35.666667,35.0,36.0,8.666667,9.0,8.0,7.000000,8.0,6.0,8.000000,8.0,8.0,8.000000,8.0,8.0,4.00,4.0,4.0,0.016393,-0.016393,0.000000,0.000000,0.000000,-0.082707,94.000000,92.000000,97.00000,-0.045329,67.560479,81.378476,...,0.016393,0.113834,74.566667,75.6,73.7,0.011475,0.002990,2.020000,2.560000,1.520000,0.602500,0.880000,0.450000,6.357500,7.460000,5.000000,70.250000,84.0,59.0,50.830000,53.040,44.20,5.268975,5.559900,5.094420,1.935125,2.56510,1.53680,280.431155,358.579514,222.155953,-0.013333,-0.005513,-0.030897,0.320513,0.113333,0.003647,-0.012169,0.234609,-0.921717
3,0,0,0.0,0.0,0.0,1.0,66.0,-462.000000,-550.000000,-0.043752,32.000000,30.0,33.0,4.250000,5.0,3.0,8.000000,8.0,8.0,8.000000,8.0,8.0,8.000000,8.0,8.0,3.75,4.0,3.0,-0.028986,0.000000,0.000000,0.000000,-0.014493,-0.057374,92.885179,89.323154,93.28186,-0.158016,82.666667,76.000000,...,-0.057971,0.000000,80.700000,81.2,79.8,-0.020290,0.002810,1.749101,2.176370,1.447025,0.492000,0.780000,0.130000,4.466000,5.480000,3.930000,166.250000,194.0,135.0,70.720000,70.720,70.72,4.887540,5.146100,4.732400,1.919320,2.29190,1.61450,278.366400,297.400000,267.660000,-0.007562,-0.006957,0.018406,0.057971,0.000000,0.000374,0.001800,-0.258609,-0.569600
4,1,0,0.0,0.0,1.0,2.0,38.0,-190.115918,-449.933130,0.024394,21.250000,19.0,24.0,9.000000,10.0,8.0,1.000000,1.0,1.0,3.750000,4.0,3.0,3.500000,5.0,2.0,4.00,4.0,4.0,0.000000,0.000000,-0.014286,0.014286,0.000000,0.009325,71.000000,60.000000,85.00000,-0.328248,110.203141,99.754150,...,0.057143,0.048566,95.050000,96.0,94.5,0.021429,0.

In [ ]:
feature_and_target_for_MI.to_csv('/content/drive/MyDrive/Colab Notebooks/예비실험 4/예비실험4 - Extract & Imputation/feature_and_target_for_MI.csv')

In [ ]:
X_data = all_feature_without_nans
Y = target

In [ ]:
X_data.to_csv('/content/drive/MyDrive/Colab Notebooks/예비실험 4/예비실험4 - Extract & Imputation/X_data_before_MI.csv')

In [ ]:
Y.to_csv('/content/drive/MyDrive/Colab Notebooks/예비실험 4/예비실험4 - Extract & Imputation/y_target.csv')